In [2]:
import datetime
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import phik
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'
sns.set_theme(context='talk', style='whitegrid', palette='deep')
plt.rcParams['figure.figsize'] = 10, 7
plt.rcParams['font.size'] = 20
plt.rcParams['axes.labelsize'] = 25
plt.rcParams['figure.titlesize'] = 32
plt.rcParams['axes.titlesize'] = 32
plt.rcParams['savefig.format'] = 'pdf'
plt.rcParams['figure.autolayout'] = 'true'
plt.rcParams['figure.frameon'] = 'false'
plt.rcParams['axes.spines.left'] = 'false'
plt.rcParams['axes.spines.right'] = 'false'
plt.rcParams['axes.spines.top'] = 'false'
plt.rcParams['legend.fancybox'] = 'false'
plt.rcParams['axes.spines.bottom'] = 'false'
plt.rcParams['font.size'] = 20
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
# для графиков, где надо много цветов, юзайте воть:
sns.set_palette(sns.color_palette('deep'))
# а по дефолту воть:
sns.set_palette(sns.color_palette('BuGn_r', n_colors=10)[2::3])
pd.set_option('display.max_columns', 60)

In [3]:
feeding_details_22 = pd.read_csv('datasets/2022-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_22 = pd.read_csv('datasets/2022-feeding-tasks.csv', on_bad_lines='skip')

feeding_details_23 = pd.read_csv('datasets/2023-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_23 = pd.read_csv('datasets/2023-feeding-tasks.csv', on_bad_lines='skip')

feeding_details_24 = pd.read_csv('datasets/2024-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_24 = pd.read_csv('datasets/2024-feeding-tasks.csv', on_bad_lines='skip')

feeding_details_25 = pd.read_csv('datasets/2025-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_25 = pd.read_csv('datasets/2025-feeding-tasks.csv', on_bad_lines='skip')

In [4]:
monthly_feeding = pd.read_excel('datasets/Ekoniva_dataset.xlsx', sheet_name='Feeding')
herd_metrics = pd.read_excel('datasets/Ekoniva_dataset.xlsx', sheet_name='Herd maintenance').replace('-', np.nan)
production_indicators = pd.read_excel('datasets/Ekoniva_dataset.xlsx', sheet_name='Dairy indicators').replace('-', np.nan)

In [5]:
def merge_tables() -> pd.DataFrame:
    """
    Merges all feeding dataframes with all details dataframes
    returns: merged dataframe
    rtype: pd.DataFrame
    """
    feeding_22["year"] = "2022"
    feeding_23["year"] = "2023"
    feeding_24["year"] = "2024"
    feeding_25["year"] = "2025"
    feeding_details_22["year"] = "2022"
    feeding_details_23["year"] = "2023"
    feeding_details_24["year"] = "2024"
    feeding_details_25["year"] = "2025"

    feeding_all = pd.concat([feeding_22, feeding_23, feeding_24, feeding_25], ignore_index=True)
    feeding_details_all = pd.concat([feeding_details_22, feeding_details_23, feeding_details_24, feeding_details_25], ignore_index=True)
    
    return feeding_all.merge(feeding_details_all, on=["FeedingTaskID", "SectionID", "year", "PhysiologicalGroupID", "PhysiologicalGroupName"], how="left"), feeding_all, feeding_details_all

feeding_and_details, feeding_all, feeding_details_all = merge_tables()


In [ ]:
import re
from collections import Counter
def group():
    def normalize_name(s):
        if pd.isna(s):
            return ""
        s = str(s).strip()
        s = s.replace("//", "/")
        s = re.sub(r"\s+", " ", s)
        s = s.lower()
        return s
    feeding_details_all['norm_ingr_name'] = feeding_details_all['IngredientName'].apply(normalize_name)
    code_re = re.compile(r'^\d+(?:\.\d+)+$') #Честно сам писал (100%)
    feeding_details_all['is_code'] = feeding_details_all['norm_ingr_name'].str.match(code_re)
    smth = pd.ExcelFile("datasets/Ekoniva_dataset.xlsx")
    nsi = smth.sheet_names[0]
    cultures = pd.read_excel(
        smth,
        sheet_name=nsi,
        usecols="B:C",
        skiprows=81,
        nrows=40 
    )
    cultures.columns = ["code", "name"]
    razdels = pd.read_excel(
        smth,
        sheet_name=nsi,
        usecols="E:H",
        skiprows=81,
        nrows=88  
    )
    razdels.columns = ["code", "region", "prop_farm_name", "farm_name"]
    feed_type = pd.read_excel(
        smth,
        sheet_name=nsi,
        usecols="B:C",
        skiprows=123,
        nrows=10
    )
    feed_type.columns = ["code", "feed_name"]
    cultures_map = cultures.set_index('code')['name'].to_dict()
    feed_type_map = feed_type.set_index('code')['feed_name'].to_dict()
    def decode_ingr_code(code):
        if pd.isna(code) or not isinstance(code, str):
            return code
        parts = code.split('.')
        if len(parts) < 4:
            return code
        try:
            culture_code = float(parts[2])
            feed_code = float(parts[3])
        except ValueError:
            return code    
        culture_name = cultures_map.get(culture_code, str(culture_code))
        feed_name = feed_type_map.get(feed_code, str(feed_code))
        return f"{culture_name} {feed_name}".lower()
    feeding_details_all['decoded_name'] = feeding_details_all.apply(
        lambda row: decode_ingr_code(row['norm_ingr_name']) if row['is_code'] else row['norm_ingr_name'],
        axis=1
    )
    GROUP_KEYWORDS = {
        "forage": [
            "силос", "сенаж", "сено", "люцерн", "солома", "зеленая масса", "зел масса", "сило", "сорго",
            "сил", "сен", "люц", "трав", "зелёная масса", "клевер", "тимофеевк", "злаковая смесь",
            "солома пшеницы", "солома ячменя", "сенокос", "сенок", "тюк", "тюкован", "травосмесь",
            "сенокуб", "сеносилос", "рожь", "з-с рож 01", "курочкино", "к-ж", "soloma", "cилос", "01.01", "02.01", "03.05", "12.01"
        ],
        "energy": [
            "кукуруза", "пшениц", "ячмень", "овёс", "кукуруза плющ", "зерно", "зerno",
            "концентрат углевод", "глюгтен", "глютен", "корнаж", "карнаж", "патока свекловичная",
            "глицерин", "патока", "с-с кукурузный 1",
            "corn", "wheat", "barley", "oats", "grain", "кук", "ячм", "овес", "пшеница", "кукурузное зерно",
            "мука кукурузная", "мука ячменная", "кукуруза молотая", "дерть", "плющенка", "кк энергетический",
            "энергетик", "карб", "крахмал", "солод", "зерносмесь", "мука пшеничная", "тритикале", "тритикале плющеная"
        ],
        "protein": [
            "шрот", "жмых", "соев", "соевый", "протеин", "оболочка сои", "соя", "люпин", "горох", "бобы",
            "белок", "лен", "дрожи",
            "шр", "соево", "соевый шрот", "rapeseed meal", "sunflower meal", "подсолн", "подсолнечный шрот",
            "шр подсолнечный", "экспандат", "экспеллер", "шрот соевый", "соевый жмых",
            "жмых рапсовый", "жмых льняной", "мучка", "мука протеиновая", "соя экструдированная",
            "bean", "peas", "protein meal", "чечевица"
        ],
        "fat": [
            "жир", "масло", "пропионат", "жир защищённый", "масло соевое",
            "триглицериды", "жир кормовой", "жир растительный", "жир животный", "жир инертный",
            "жир корм", "пальмовое масло", "РЖК", "protected fat", "жирный концентрат"
        ],
        "premix": [
            "премикс", "премикс дойн", "премикс молод", "премикс транзит",
            "premix", "пре", "мин-премикс", "вит-премикс", "премикс для коров",
            "премикс для телят", "премикс уни", "пр.дойный"
        ],
        "mineral_vit": [
            "мел", "соль", "витамин", "минерал", "извест", "кальций", "фосфат",
            "лизунец", "минвит", "сода", "поташ", "галит брикеты",
            "Ca", "P", "NaCl", "CaCO3", "минсмесь", "витамины", "минералы",
            "ди кальций фосфат", "морская соль", "лизун", "лизунцы", "соль кормовая",
            "сода корм", "кальцит", "лед",  "soda"
        ],
        "additive": [
            "дрожж", "пробиот", "фермент", "энзим", "энзимы", "буфер", "аквасейф", "биотек",
            "мегабуст", "электролит", "кисол", "шаумацид дринк",
            "пробиотик", "бактерии", "закваска", "пребиотик", "аминокислота",
            "сорбент", "адсорбент", "консервант", "антиоксидант", "детокс", "детоксикант",
            "биостимулятор", "acid", "enzyme", "холин", "провилит", "ампролиум"
        ],
        "byproduct": [
            "жом", "барда", "жом свекловичный", "сыворотка", "дробина", "зeрноотход",
            "остатк", "остатки",
            "пивная дробина", "барда сухая", "жом сухой", "жом гранулированный",
            "DDGS", "ддгс", "зерноотходы", "побочка", "отруби", "брикет жома",
            "меласса", "oстатки", "сухая стружка"
        ],
        "water": [
            "вода", "влажн", "сыворотка молочная", "молоко",
            "H2O", "жидкость", "влажное сырье", "сыворотк", "сыв", "обрат"
        ],
        "premix_blend": [
            "комбикорм", "кк", "кк№", "кк №", "комбикорм №",
            "стартер", "престартер", "бустер милк", "кормосмесь №10 снт",
            "кальвобустер", "бвмк для коров", "зцм",
            "ККР", "концентрат", "смесь", "БВМК", "ПК", "концентрат белковый",
            "концентрат энергетический", "комби", "смесь кормовая",
            "complete feed", "feed mix", "starter feed", "grower", "зск", "бвмк", "kkd1"
        ]
    }
    def classify_ingredient(name):
        if not isinstance(name, str):
            return "other"
        name_lower = name.lower()
        for group, kat_list in GROUP_KEYWORDS.items():
            for kat in kat_list:
                if kat in name_lower:
                    return group
        return "other"
    feeding_details_all['ingredient_group'] = feeding_details_all['decoded_name'].apply(classify_ingredient)
    feeding_details_all = feeding_details_all[feeding_details_all["ingredient_group"] != "other"]


group()


Тест на каузальность грейнджера

In [8]:
feeding_details_all.ingredient_group.value_counts()

ingredient_group
forage          5963982
energy          2873289
protein         2664047
water            971809
byproduct        962749
premix_blend     943903
premix           935655
mineral_vit      672546
fat              264034
additive         191061
other                 4
Name: count, dtype: int64

In [95]:
others = feeding_details_all[feeding_details_all['ingredient_group'] == 'other']
others['decoded_name'].value_counts()

decoded_name
    4
Name: count, dtype: int64

In [ ]:
others[others["decoded_name"] == ""]

,FeedingTaskID,SectionID,PhysiologicalGroupID,PhysiologicalGroupName,IngredientID,IngredientName,IngredientType,PhysicalWeight_kg,year,norm_ingr_name,is_code,decoded_name,ingredient_group
11574693,Farms/EkoNiva1C.9d4645a7-570c-11e2-9cc0-00155d...,342,54,Т1 (3-5 мес.),260,NaN,Premixture,0.0,2024,,False,,other
11592293,Farms/EkoNiva1C.9d4645a7-570c-11e2-9cc0-00155d...,342,54,Т1 (3-5 мес.),260,NaN,Premixture,0.0,2024,,False,,other
11608364,Farms/EkoNiva1C.9d4645a7-570c-11e2-9cc0-00155d...,342,54,Т1 (3-5 мес.),260,NaN,Premixture,0.0,2024,,False,,other
11622461,Farms/EkoNiva1C.9d4645a7-570c-11e2-9cc0-00155d...,342,54,Т1 (3-5 мес.),260,NaN,Premixture,0.0,2024,,False,,other


In [109]:
feeding_details_all

,FeedingTaskID,SectionID,PhysiologicalGroupID,PhysiologicalGroupName,IngredientID,IngredientName,IngredientType,PhysicalWeight_kg,year,norm_ingr_name,is_code,decoded_name,ingredient_group
0,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,125,Солома покупная,Forage,150.259,2022,солома покупная,False,солома покупная,forage
1,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,773,3645.01.01.01.1.20,Forage,880.444,2022,3645.01.01.01.1.20,True,люцерна сенаж,forage
2,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,72,Комбикорм 10 группы,Concentrate,542.257,2022,комбикорм 10 группы,False,комбикорм 10 группы,premix_blend
3,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,82,Кукуруза сухая,Concentrate,991.793,2022,кукуруза сухая,False,кукуруза сухая,energy
4,Farms/EkoNiva1C.216d4235-2852-11e8-80c4-1c98ec...,9,2,Д1,129,Шрот подсолнечный,Concentrate,156.054,2022,шрот подсолнечный,False,шрот подсолнечный,protein
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16443074,Farms/EkoNiva1C.6c69fe1b-2b6f-11e8-80c4-1c98ec...,131,1,Нетели,119,Солома (общ.),Forage,120.643,2025,солома (общ.),False,солома (общ.),forage
16443075,Farms/EkoNiva1C.6c69fe1b-2b6f-11e8-80c4-1c98ec...,131,1,Нетели,335,6203.01.01.01.1.25,Forage,549.169,2025,6203.01.01.01.1.25,True,люцерна сенаж,forage
16443076,Farms/EkoNiva1C.6c69fe1b-2b6f-11e8-80c4-1c98ec...,131,1,Нетели,331,Жом свекловичный сухой,Concentrate,36.676,2025,жом свекловичный сухой,False,жом свекловичный сухой,byproduct
16443077,Farms/EkoNiva1C.6c69fe1b-2b6f-11e8-80c4-1c98ec...,131,1,Нетели,143,Премикс молодняк 6-24,VitaminMineral,10.049,2025,премикс молодняк 6-24,False,премикс молодняк 6-24,premix


In [112]:
feeding_details_all = feeding_details_all[feeding_details_all["ingredient_group"] != "other"]

In [113]:
feeding_details_all.to_csv("out.csv")